In [28]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import types
import time
import re
import queue
import string

In [29]:
#Get Seed URLs and URLs from the seed URL
def getQueue(url_lst):
    q = queue.Queue()
    for url in url_lst:
        r = requests.get("http://" + url)
        data = r.text
        soup = BeautifulSoup(data,"lxml")
        for link in soup.find_all('a'):
            link = link.get('href')
            if link!=None:
                q.put(link)
    return q

In [30]:
#Determine if page is relevant (before downloading)
def classRel(url,html,rel_list):
    soup = BeautifulSoup(html,"lxml")
    text = soup.get_text()
    text=text.lower()
    rel_count=0
    for item in rel_list:
        item=item.lower()
        if item in text:
            rel_count += 1
        else:
            continue
    if rel_count>=2:
        return True
    else:
        return False

In [31]:
#Make sure the url is valid before crawling
def makeVal(url):
    if 'redlink' in url:
        return None
    name = getName(url)
    name = re.sub("[^a-zA-Z]","",name)
    if name == None:
        return None
    try:
        if '#' in url:
            return None
        elif url[1]=='w':
            url = "http://en.wikipedia.org" + url
            return url
        else:
            return None
    except IndexError:
        print ("bad page name")

In [32]:
#Create a valid file name for the downloaded page
def getName(url):
    name = url.split('/')[-1]
    if '.' in name:
        name = name.split('.')[0]
    return name

In [33]:
#Download pages from the urls in the queue
def getPage(url,rel_list):
    html = urlopen(url).read()
    path="Crawled_Pages/"
    name = getName(url)
    try:
        if classRel(url,html,rel_list):
            file = open(path+name+'.html','w')
            file.write(str(html))
            file.close()
    except OSError:
        print ('bad page')

In [34]:
#Get the relevant search terms from the user
def getRelList():
    rel_list = []
    counter = 0
    b = True
    while b==True:
        term = input('Enter a search term(Leave blank to end): ')
        if term=="" and counter>=2:
            b=False
        elif term=="" and counter<2:
            print('You need at least 2 search terms')
            continue
        else:
            rel_list.append(term)
            counter+=1
    return rel_list

In [35]:
#Get the Seed URLs fromt the user
def getSeeds():
    seed_URLs = []
    counter = 0
    b = True
    while b==True:
        term = input('Enter a seed URL(Leave blank to end):')
        if term=="" and counter>=1:
            b=False
        elif term=="" and counter<1:
            print('You need at least 1 seed URL')
            continue
        else:
            seed_URLs.append(term)
            counter+=1
    return seed_URLs

In [36]:
#seed_URLs = ['en.wikipedia.org/wiki/Hinduism','en.wikipedia.org/wiki/India']
#rel_list = ['Hindi','Mughal','Vishnu','muslim','cricket','modi','pakistan','Indus','dharma','british']
seed_URLs = getSeeds()
rel_list = getRelList()
q=getQueue(seed_URLs)
vis_URL = []
while not q.empty():
    url = q.get()
    url = makeVal(url)
    if url != None and url not in vis_URL:
        getPage(url,rel_list)
        vis_URL.append(url)
    else:
        continue

Enter a seed URL(Leave blank to end):en.wikipedia.org/wiki/Hinduism
Enter a seed URL(Leave blank to end):en.wikipedia.org/wiki/India
Enter a seed URL(Leave blank to end):
Enter a search term(Leave blank to end): mughal
Enter a search term(Leave blank to end): muslim
Enter a search term(Leave blank to end): cricket
Enter a search term(Leave blank to end): pakistani
Enter a search term(Leave blank to end): Indus
Enter a search term(Leave blank to end): dharma
Enter a search term(Leave blank to end): 
